# From Experiments to Data
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023
By: Per B. Sederberg, PhD



# Lesson Objectives

Upon completion of this lesson, students should have learned:

1. How to polish up a complete experiment
2. How SMILE stores data
3. How to read in slog files
4. An introduction to Pandas


# Review list generation solution

- Let's go through the list gen code you'll use for your next assignment!

# Subject Info

- When collecting data from a lot of participants in a lab setting, you need an easy way to enter subject information once the experiment has started.

In [9]:
from smile.common import *
from smile.startup import InputSubject

exp = Experiment(show_splash=False, resolution=(1024,768), 
                 scale_up=True, scale_down=True, 
                 scale_box=(1000, 750))

InputSubject('Flanker')

Label(text='Hello!')
with UntilDone():
    KeyPress()
    
exp.run()

# Instructions

- Participants need some guidance as to how they should perform the task
- This can be just text via `Label` states, but even better would be a visual description, and even better would be a full tutorial.
- ***NOTE: If you provide a tutorial, do not use items that you will use for the actual task!***

In [2]:
from smile.common import *

exp = Experiment(show_splash=False, resolution=(1024,768))

# Labels can be multi-line with markup!
with Parallel():
    Label(text="This is multi-line text aligned to the left", 
          font_size=50, text_size=(600, None),
          left=exp.screen.left)
    Label(text="This is multi-line text aligned to the right", 
          font_size=50, text_size=(600, None), 
          right=exp.screen.right, halign='right')
with UntilDone():
    KeyPress()
    
exp.run()

# Scaling to different screen sizes/densities

- Screen sizes and pixel densities can vary across devices (especially on laptops, tablets, and phones)
- But all sizes in Kivy/SMILE are defined by number of pixels
- We provide a means to scale across devices:

In [12]:
# load in smile states
from smile.common import *
from smile.scale import scale as s

# create an experiment instance (pick different resolutions)
resolution = (400,300)
#resolution = (800,600)
#resolution = (1024,768)
exp = Experiment(show_splash=False, resolution=resolution,
                 scale_up=True, scale_down=True, 
                 scale_box=(400, 300))
Wait(.5)
with Parallel():
    # add a scaled rectangle
    Rectangle(width=s(250),height=s(250), color='red')
    
    # and a fixed size rectangle
    rect = Rectangle(width=100, height=100, color='blue')
    
    Label(text="Blue", bottom=rect.top, fontsize=s(50))
with UntilDone():
    KeyPress()

exp.run()

# Logging and Slogging

- SMILE states automatically log themselves, so you rarely will lose information if you forget to log it.
- It's still much easier to analyze a well-organized log file from your experiment.
- Make use of the `Log` state to save out data.
- All logs are stored in `.slog` files, which is short for SMILE Log.
- These are read in as a list of dictionaries (or a dict-list):

```python
from smile.log import log2dl
dl = log2dl('flanker_log.slog')
```

# Where have all the data gone?

- All the data files are stored in the `data` directory
  - NOTE: Even when you don't specify a subj, it saves to a `test000` directory, so you can delete that sometimes to save space.
- There is a hierarchical structure to the data directory:
  - Experiment -> SubjID -> Date/Time

In [7]:
!du -sh data

1.4M	data


In [6]:
!ls data/SMILE/test000/

20230907_002217  20230907_150037  20230914_104602  20230914_145307
20230907_002319  20230907_160531  20230914_104621  20230914_145716
20230907_004819  20230907_160546  20230914_111058  20230914_150053
20230907_004823  20230907_160610  20230914_111249  20230914_151750
20230907_004836  20230907_161810  20230914_112306  20230914_152130
20230907_004858  20230907_162329  20230914_112316  20230914_152644
20230907_103848  20230907_162413  20230914_112524  20230914_153137
20230907_103925  20230907_162421  20230914_113326  20230914_153155
20230907_104009  20230907_162436  20230914_114139  20230914_153241
20230907_113136  20230907_162448  20230914_114452  20230914_153354
20230907_113245  20230914_092020  20230914_114640  20230914_160001
20230907_113319  20230914_092225  20230914_114906  20230917_131051
20230907_113329  20230914_092251  20230914_114916  20230917_131127
20230907_113420  20230914_092304  20230914_134250  20230917_131139
20230907_113705  20230914_092353  20230914_134422  20230920_23

In [13]:
from smile.log import log2dl
dl = log2dl('data/SMILE/test000/20230920_234838/state_KeyPress_0.slog')
dl

[{'instantiation_filename': 'Turn on Debug Mode for this information',
  'instantiation_lineno': 0,
  'name': None,
  'start_time': 1411736.494129947,
  'end_time': 1411738.4637933115,
  'enter_time': 1411736.249198686,
  'leave_time': 1411738.464830402,
  'finalize_time': 1411738.464898925,
  'base_time': 1411736.494129947,
  'pressed': 'F',
  'press_time_time': 1411738.4637933115,
  'press_time_error': 0.0006595384329557419,
  'correct': False,
  'rt': 1.969663364579901,
  'log_num': 0}]

# Let's continue learning by building together!

- Remaining features to add:
  - Instructions
  - Loop over blocks and then trials
  - Verify logging

# List Gen Function

- NOTE: New addition for multiple blocks of trials

In [14]:
import random 
import copy

# define the conditions
conds = [{'condition': 'congruent',
          'direction': 'left',
          'stimulus': '<<<<<<<'
         },
         {'condition': 'congruent',
          'direction': 'right',
          'stimulus': '>>>>>>>'
         },
         {'condition': 'incongruent',
          'direction': 'left',
          'stimulus': '>>><>>>'
         },
         {'condition': 'incongruent',
          'direction': 'right',
          'stimulus': '<<<><<<'
         },
         {'condition': 'neutral',
          'direction': 'left',
          'stimulus': '===<==='
         },
         {'condition': 'neutral',
          'direction': 'right',
          'stimulus': '===>==='
         },]

# specify number of reps of these conditions
num_reps = 2
num_blocks = 3

# loop and create the blocks
blocks = []
for b in range(num_blocks):
    # loop and create the list
    trials = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        trials.extend(copy.deepcopy(conds))
        
    # shuffle the trials
    random.shuffle(trials)
    
    # append the trials to the blocks
    blocks.append(trials)

print(blocks)

[[{'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}, {'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}, {'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}, {'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}], [{'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'neutral', 'direction': 'left', 'stimul

# Goal for task

## High level structure

- Present instructions
- Loop over blocks
- Within each block, loop over trials

## Trial structure

- Present the correct stimulus as text on the screen
- Wait for a response
- Remove the stimulus
- Wait for an inter-stimulus interval
- Log the result of the trial

In [15]:
from smile.common import *
from smile.scale import scale as s

# config section
font_size = 75
resp_keys = ['F', 'J']
resp_map = {'left': 'F', 'right': 'J'}
ISI_dur = 0.5
ISI_jitter = 0.5
LOC_X_jitter = 200
LOC_Y_jitter = 100
inst_font_size = 25
inst_text = """[u][size=40]FLANKER INSTRUCTIONS[/size][/u]

In this task, you will see stimuli one at at time on the screen. 
    
Press ENTER key to continue."""

# create the experiment
exp = Experiment(name='FLANKER', show_splash=False, 
                 fullscreen=False,
                 resolution=(1024, 768), scale_box=(1024, 768))

@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None),
          markup=True)
    with UntilDone():
        KeyPress(keys=['ENTER'])

@Subroutine
def Trial(self, block_num, trial_num, cur_trial):
    # pick the new stimulus location
    self.location = (jitter(self.exp.screen.center_x-LOC_X_jitter,
                            LOC_X_jitter*2),
                     jitter(self.exp.screen.center_y-LOC_Y_jitter,
                            LOC_Y_jitter*2))
    # present the stimulus
    stim = Label(text=cur_trial['stimulus'],
                 font_size=font_size,
                 center=self.location)
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        
        # collect a response (with no timeout)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[cur_trial['direction']])
    
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)

    # log the result of the trial
    Log(name='flanker', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
        location=self.location
       )

# Get the subj id information


# show the instructions
Instruct()
Wait(0.5)

# loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])

    # add in some delay before the start of the block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # loop over the trials
    with Loop(block.current) as trial:
        Trial(block.i, trial.i, trial.current)

# make sure they are ready to continue
Label(text='You are all done!!!\nPress the ENTER key to go celebrate.', 
      font_size=font_size, halign='center')
with UntilDone():
    KeyPress(keys=['ENTER'])

# run the experiment
exp.run()

# Read in some data from the logs

In [16]:
from smile.log import log2dl
dl = log2dl('data/FLANKER/test000/20230921_120950/log_flanker_0.slog')
dl

[{'block_num': 0,
  'trial_num': 0,
  'stim_on_time': 1427074.569719217,
  'stim_on_error': 0.0,
  'resp': 'F',
  'resp_time_time': 1427075.304962115,
  'resp_time_error': 0.00022440101020038128,
  'rt': 0.7352428978774697,
  'correct': True,
  'location_0': 377.527008863294,
  'location_1': 377.9792209665405,
  'log_time': 1427076.2610206704,
  'condition': 'neutral',
  'direction': 'left',
  'stimulus': '===<===',
  'log_num': 0},
 {'block_num': 0,
  'trial_num': 1,
  'stim_on_time': 1427076.275113663,
  'stim_on_error': 0.0,
  'resp': 'F',
  'resp_time_time': 1427077.013456764,
  'resp_time_error': 0.00022043799981474876,
  'rt': 0.7383431009948254,
  'correct': True,
  'location_0': 710.0734405646895,
  'location_1': 480.5401723545676,
  'log_time': 1427078.0035128118,
  'condition': 'congruent',
  'direction': 'left',
  'stimulus': '<<<<<<<',
  'log_num': 0},
 {'block_num': 0,
  'trial_num': 2,
  'stim_on_time': 1427078.011423538,
  'stim_on_error': 0.0,
  'resp': 'J',
  'resp_tim

In [18]:
import pandas as pd

df = pd.DataFrame(dl)
df.head()

,block_num,trial_num,stim_on_time,stim_on_error,resp,resp_time_time,resp_time_error,rt,correct,location_0,location_1,log_time,condition,direction,stimulus,log_num
0,0,0,1.427075e+06,0.0,F,1.427075e+06,0.000224,0.735243,True,377.527009,377.979221,1.427076e+06,neutral,left,===<===,0
1,0,1,1.427076e+06,0.0,F,1.427077e+06,0.000220,0.738343,True,710.073441,480.540172,1.427078e+06,congruent,left,<<<<<<<,0
2,0,2,1.427078e+06,0.0,J,1.427079e+06,0.000473,0.609006,True,367.079417,410.745729,1.427079e+06,congruent,right,>>>>>>>,0
3,0,3,1.427079e+06,0.0,F,1.427080e+06,0.000217,0.817039,True,590.853489,425.856978,1.427081e+06,incongruent,left,>>><>>>,0
4,0,4,1.427081e+06,0.0,F,1.427082e+06,0.000243,0.866920,True,318.416581,475.675454,1.427082e+06,incongruent,left,>>><>>>,0


## Assignment before and after next class

- Your memory list generation is due Sunday by midnight!
- Start thinking about the SMILE experiment code. I will release the list generation solution on Monday. The SMILE experiment will be due two weeks from today (Thursday).
- Run in the Flanker experiment we will make available and upload your data by Tuesday night at midnight.

### See you next week!!!